In [1]:
import matplotlib.pyplot as plt
from perlin_noise import PerlinNoise
import time
import random
import os
import pathlib
import numpy as np
from PIL import Image
# from descartes import PolygonPatch
# import alphashape
#import cartopy.crs as ccrs
from scipy.spatial import ConvexHull

import cv2
#import warning

In [2]:
def centerMask(heightMap):
    center = (125, 125)
    radius = 15
    
    x = np.arange(0, 250)
    y = np.arange(0, 250)
    mask = (x[np.newaxis,:]-center[0])**2 + (y[:,np.newaxis]-center[1])**2 < radius**2
    
    return heightMap*mask, mask

In [3]:
def primitiveCircle(center : tuple, radius : int, heightMap):
    x = np.arange(0, 250)
    y = np.arange(0, 250)

    # The two lines below could be merged, but I stored the mask
    # for code clarity.
    mask = (x[np.newaxis,:]-center[0])**2 + (y[:,np.newaxis]-center[1])**2 < radius**2
    heightMap[mask] = 255 # * random.randrange(2)
    

    return heightMap

In [118]:
def rectangle(heightMap):
    mask = np.zeros((250, 250), dtype=np.uint8)
    x=random.randrange(0+10,250-10)
    y=random.randrange(0+10,250-10)
    width=random.randrange(10,15)
    height=random.randrange(10,15)
    angle=random.randrange(0,45)
    rect = plt.Rectangle((x,y), width, height, angle=angle)
    points = rect.get_verts().astype(np.int32)
    
    cv2.fillPoly(mask, points[np.newaxis,:], (1)) #new axis is to get in correct array format
    isExtrusion = 1 # random.randrange(2)
    
    return (heightMap*(1-mask) + np.uint8(255*(mask)*isExtrusion))
#     return points

In [114]:
def convex(heightMap):
    mask = np.zeros((250, 250), dtype=np.uint8)
    numPoints = random.randrange(4,7)
    points = np.array([[random.randrange(250), random.randrange(250)]])
    for i in range(numPoints):
        xdelta = random.randrange(-10,10)
        ydelta = random.randrange(-10,10)
        points = np.append(points, points[i,:] + [[xdelta, ydelta]], axis=0)
    points=(np.int32(points))
    pvs=ConvexHull(points).vertices
    points=points[pvs]
    cv2.fillPoly(mask, points[np.newaxis,:], (1)) #new axis is to get in correct array format
    isExtrusion = 1 # random.randrange(2)
    
    return (heightMap*(1-mask) + np.uint8(255*(mask)*isExtrusion))
#     return points

In [5]:
def nonConvex(heightMap):
    mask = np.zeros((250, 250), dtype=np.uint8)
    numPoints = random.randrange(4,7)
    points = np.array([[random.randrange(250), random.randrange(250)]])
    for i in range(numPoints):
        xdelta = random.randrange(-10,10)
        ydelta = random.randrange(-10,10)
        points = np.append(points, points[i,:] + [[xdelta, ydelta]], axis=0)
    points=(np.int32(points))
    cv2.fillPoly(mask, points[np.newaxis,:], (1)) #new axis is to get in correct array format
    isExtrusion = 1 # random.randrange(2)
    
    return (heightMap*(1-mask) + np.uint8(255*(mask)*isExtrusion))

In [ ]:
# Parameters
startImages = 5001
endImages = 5002

xpix, ypix = 250, 250
imageScale = 0.5

octaves = 1

obstacleName = "nonconvex"
if obstacleName == "rectangle":
    maxRadius = 0
    minRadius = 0
    minPrimCircle = 0
    maxPrimCircle = 0
    minNonConvex = 0
    maxNonConvex = 0
elif obstacleName == "circle":
    maxRadius = 10
    minRadius = 5
    minPrimCircle = 7
    maxPrimCircle = 15
    minNonConvex = 0
    maxNonConvex = 0
elif obstacleName == "ramp":
    print("ok")
elif obstacleName == "nonconvex":
    maxRadius = 0
    minRadius = 0
    minPrimCircle = 0
    maxPrimCircle = 0
    minNonConvex = 7
    maxNonConvex = 15
elif obstacleName == "all":
    print("ok")
else:
    raise Exception("Not valid obstacle name")
    
    
path=os.getcwd()
folder_path=path+(f'/heightmaps/%s/train data/inputs/' % obstacleName)


try:
    os.makedirs(folder_path)
except:
    print("Directory already exists...")

# data_dir=pathlib.Path(path+'/heightmaps/octaves 3/test data/inputs')
random.seed(int(time.time()))

prevTime = time.time()
for img_num in range(startImages, endImages):
    seed = random.randrange(100000000)
    noise = PerlinNoise(octaves=octaves, seed=seed)
    pic = [[noise([i/xpix, j/ypix]) for j in range(xpix)] for i in range(ypix)]
    
    pic_np = (np.array(pic)).reshape((xpix,ypix))
    p_max = np.max(pic_np)
    p_min = np.min(pic_np)
    if p_max-p_min>1:
        pic_np = (((pic_np-p_min)/(p_max-p_min))*imageScale*255).astype('uint8')
    else: pic_np = ((pic_np-p_min)*imageScale*255).astype('uint8')
    pic_np = (pic_np*0 + 25).astype('uint8') # Reset map to flat terrain
    
    pic_np_center, center_mask = centerMask(pic_np)
    
    if maxPrimCircle > 0:
        numOfPrimCircle = random.randrange(minPrimCircle, maxPrimCircle)
        for i in range(numOfPrimCircle):
            #print(randCenter)
            randCenter = (random.randrange(xpix), random.randrange(ypix))
            randRadius = random.randrange(minRadius, maxRadius)
            pic_np = primitiveCircle(randCenter, randRadius, pic_np)
    if maxNonConvex > 0:
        numOfNonConvex = random.randrange(minNonConvex, maxNonConvex)
        for j in range(numOfNonConvex):
            pic_np = nonConvex(pic_np)
    
    pic_np = np.uint8(pic_np*(1-center_mask) + pic_np_center)
    
    pil_img = Image.fromarray(pic_np, 'L')
    # img_path=folder_path+obstacleName[0]+'_H'+str(img_num+1)+'.png'
    img_path=folder_path+str(img_num+1)+'.png'
    pil_img.save(img_path)
    
    
    if not (img_num+1)%200:
        print("Number of Terrains Generated:",img_num+1)
        currentTime = time.time()
        print("Duration: %0.3fs" % (currentTime-prevTime))
        prevTime = currentTime
    
print("<<<<<<<<<<<<<<<//////////////////// Done Generating Terrains \\\\\\\\\\\\\\\\\\\\\\\>>>>>>>>>>>>>>>>>>>>")

In [ ]:
print(folder_path)